In [80]:
import pandas as pd
import pyautogui
import time
import pyperclip
import openpyxl


# Funções

In [81]:
def encontrar_imagem(imagem):
    while not pyautogui.locateOnScreen(imagem, confidence=0.8):
        time.sleep(1)
    return pyautogui.locateOnScreen(imagem, confidence=0.8)


def converter_num_Brasil(num):
    return num.replace(",", "_").replace(".", ",").replace("_", ".")

def copiar_colar(texto):
    pyperclip.copy(texto)
    pyautogui.hotkey("ctrl", "v")
    return texto

    

# Tratando planilha Declaração Pessoal

In [82]:
ano = 2022

#Importando e tratando a planilha de Declaração

rend_isentos = pd.read_excel("Declaração_Pessoal.xlsx", sheet_name = "Rend. Isentos e Não Trib.", dtype={"Tipo de Rendimento": str})
rend_trib_excl = pd.read_excel("Declaração_Pessoal.xlsx", sheet_name = "Rend. Sujeitos A Trib. Exclusiv", dtype={"Tipo de Rendimento": str})
pagamentos_efetudos = pd.read_excel("Declaração_Pessoal.xlsx", sheet_name = "Pagamentos Efetuados", dtype={"Código": str})
bens_direitos = pd.read_excel("Declaração_Pessoal.xlsx", sheet_name = "Bens e Direitos", dtype={"Código": str, "Grupo": str})
base_ticket = pd.read_excel("Declaração_Pessoal.xlsx", sheet_name = "Base", dtype={"Código": str, "Grupo": str})
avenue = pd.read_excel("Declaração_Pessoal.xlsx", sheet_name= "Avenue", dtype={"Código": str, "Grupo": str})

#Localizando o hifem e criando uma coluna para apenas as letras do código
for linha in base_ticket.index:
    posicao_1 = base_ticket.loc[linha, "Código de Negociação"].find("1")
    if posicao_1 != -1:
        if posicao_1 >= 1 and base_ticket.loc[linha, "Código de Negociação"][posicao_1] == "1":
            base_ticket.loc[linha, "inicio_ticket"] = base_ticket.loc[linha, "Código de Negociação"][:posicao_1]
        else:
            base_ticket.loc[linha, "inicio_ticket"] = base_ticket.loc[linha, "Código de Negociação"]
    else:
        base_ticket.loc[linha, "inicio_ticket"] = base_ticket.loc[linha, "Código de Negociação"][:-1]



C:\Users\guilh\AppData\Local\Temp\ipykernel_14992\3504581817.py:21: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'BBAS' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  base_ticket.loc[linha, "inicio_ticket"] = base_ticket.loc[linha, "Código de Negociação"][:-1]


# Tratando planilha de Negociações

In [83]:
base_negociacao_df = pd.read_excel("Negociação.xlsx")
base_negociacao_df = base_negociacao_df.drop(["Mercado", "Prazo/Vencimento", "Instituição"], axis=1)

# Criando dataframes para filtrar os Tipo de Movimentação e  retirando o F dos códigos que terminam com ele
base_negociacao_df_compras = base_negociacao_df.loc[base_negociacao_df["Tipo de Movimentação"] == "Compra"]
base_negociacao_df_vendas = base_negociacao_df.loc[base_negociacao_df["Tipo de Movimentação"] == "Venda"]
base_negociacao_df.loc[base_negociacao_df["Código de Negociação"].str.endswith('F'), "Código de Negociação"] = base_negociacao_df["Código de Negociação"].str.rstrip('F')

#Criando um dataframe apenas para vendas e outro para compras
compras_df = pd.DataFrame(columns=["Código de Negociação", "Valor", "Quantidade"])
vendas_df = pd.DataFrame(columns=["Código de Negociação", "Valor", "Quantidade"])

compras_df = pd.concat([compras_df, base_negociacao_df_compras])
vendas_df = pd.concat([vendas_df, base_negociacao_df_vendas])

#Agrupando por Código de Negociação e somando os respectivos valores totais
compras_df = compras_df.groupby("Código de Negociação").agg({'Valor': 'sum'}).reset_index()
vendas_df = vendas_df.groupby("Código de Negociação").agg({'Valor': 'sum'}).reset_index()

vendas_df.loc[vendas_df["Código de Negociação"].str.endswith('F'), "Código de Negociação"] = vendas_df["Código de Negociação"].str.rstrip('F')
compras_df.loc[compras_df["Código de Negociação"].str.endswith('F'), "Código de Negociação"] = compras_df["Código de Negociação"].str.rstrip('F')

# Combinando os DataFrames de compras e vendas em um único DataFrame usando merge
resultado_df = pd.merge(compras_df, vendas_df, on="Código de Negociação", suffixes=("_compra", "_venda"), how="left")
# Preenchendo os valores ausentes (NaN) em "Valor_venda" com 0
resultado_df["Valor_venda"].fillna(0, inplace=True)


#Localizando o primeiro 1 e criando uma coluna para apenas as letras do código
for linha in resultado_df.index:
    posicao_1 = resultado_df.loc[linha, "Código de Negociação"].find("1")
    if posicao_1 != -1:
        if posicao_1 >= 1 and resultado_df.loc[linha, "Código de Negociação"][posicao_1 - 3] == "1":
            resultado_df.loc[linha, "inicio_ticket"] = resultado_df.loc[linha, "Código de Negociação"][:posicao_1 - 2]
        else:
            resultado_df.loc[linha, "inicio_ticket"] = resultado_df.loc[linha, "Código de Negociação"][:posicao_1]
    else:
         resultado_df.loc[linha, "inicio_ticket"] = resultado_df.loc[linha, "Código de Negociação"][:-1]

resultado_df = resultado_df.groupby(["inicio_ticket"]).agg({'Valor_compra': 'sum', 'Valor_venda': 'sum'}).reset_index()


C:\Users\guilh\AppData\Local\Temp\ipykernel_14992\1943051171.py:13: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  compras_df = pd.concat([compras_df, base_negociacao_df_compras])
C:\Users\guilh\AppData\Local\Temp\ipykernel_14992\1943051171.py:14: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  vendas_df = pd.concat([vendas_df, base_negociacao_df_vendas])
C:\Users\guilh\AppData\Local\Temp\ipykernel_14992\1943051171.py:38: FutureWarning: Setting an item of incompatible dtype is deprecated and will r

# Tratando planilha de Posições dos Ativos

In [84]:
posicao_ativos_df = pd.read_excel("Relatorio Consolidado.xlsx", sheet_name = [0,1,2,3,4])
posicao_ativos_df = pd.concat(posicao_ativos_df.values(), ignore_index=True) 
posicao_ativos_df = posicao_ativos_df.drop(["Modalidade","Quantidade Disponível", "Administrador", "Instituição", "Conta", "Código ISIN / Distribuição", "Tipo", "Escriturador", "Quantidade Indisponível", "Motivo", "Natureza", "Número de Contrato", "OPA", "Liquidação antecipada", "Taxa", "Comissão", "Data de registro", "Data de vencimento"], axis =1)

#Localizando a posição do hifem e criando a coluna para as letras dos códigos
for linha in posicao_ativos_df.index:
    posicao_hifem = posicao_ativos_df.loc[linha, "Produto"].find("-")
    if posicao_hifem != -1:
        posicao_ativos_df.loc[linha, "Razão Social"] = posicao_ativos_df.loc[linha, "Produto"][posicao_hifem + 1:].strip()
        if posicao_hifem >= 1 and posicao_ativos_df.loc[linha, "Produto"][posicao_hifem - 2] == "1":
            posicao_ativos_df.loc[linha, "inicio_ticket"] = posicao_ativos_df.loc[linha, "Produto"][:posicao_hifem - 3]
        else:
            posicao_ativos_df.loc[linha, "inicio_ticket"] = posicao_ativos_df.loc[linha, "Produto"][:posicao_hifem - 2]

razao_social = pd.DataFrame(posicao_ativos_df[["Razão Social", "inicio_ticket"]])
           
#Agrupando os ativos que possuem mesma letra         
posicao_ativos_df = posicao_ativos_df.groupby(["inicio_ticket"]).agg({'Quantidade': 'sum', 'Valor Atualizado': 'sum'}).reset_index()

#Em caso de duplicação da coluna quantidade, remove -las
columns_to_remove = ["Quantidade_x", "Quantidade_y"]
for col in columns_to_remove:
    if col in resultado_df.columns:
        resultado_df = resultado_df.drop(col, axis=1)



#Passando a coluna quantidade da posição atual para resultado
resultado_df = resultado_df.merge(posicao_ativos_df[["inicio_ticket", "Quantidade"]], on="inicio_ticket", how="left")

# Calculando o "Preço Médio" diretamente no DataFrame resultante

resultado_df["Preço Médio"] = (resultado_df["Valor_compra"] - resultado_df["Valor_venda"]) / (resultado_df["Quantidade"])
resultado_df["Preço Médio"] = resultado_df["Preço Médio"].apply(lambda x: f"{x: .2f}")
#Juntando o Preço Médio no dataframe de Posição dos Ativos
posicao_ativos_df = posicao_ativos_df.merge(resultado_df[["inicio_ticket", "Preço Médio"]], on = "inicio_ticket", how = "inner")
posicao_ativos_df = posicao_ativos_df.merge(razao_social[["inicio_ticket", "Razão Social"]], on = "inicio_ticket", how = "inner")
posicao_ativos_df = posicao_ativos_df.merge(base_ticket[["inicio_ticket", "Código de Negociação", "Grupo", "Código", "Valor Ano Anterior", "CNPJ Formatado"]], on = "inicio_ticket", how = "inner")
posicao_ativos_df = posicao_ativos_df.drop_duplicates(subset=["inicio_ticket"])




C:\Users\guilh\AppData\Local\Temp\ipykernel_14992\3150339327.py:9: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'AMBEV S.A.' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  posicao_ativos_df.loc[linha, "Razão Social"] = posicao_ativos_df.loc[linha, "Produto"][posicao_hifem + 1:].strip()
C:\Users\guilh\AppData\Local\Temp\ipykernel_14992\3150339327.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'ABEV' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  posicao_ativos_df.loc[linha, "inicio_ticket"] = posicao_ativos_df.loc[linha, "Produto"][:posicao_hifem - 2]


# Rendimentos Isentos e Não Tributavéis

In [85]:
pyautogui.alert("O código iniciará. Não mexer em NADA no computador enquanto o código estiver rodando. Assim que finalizar eu te aviso.")
pyautogui.FAILSAFE = True
pyautogui.PAUSE = 0.5


imagem = encontrar_imagem("rend_isentos.png")
pyautogui.click(pyautogui.center(imagem))

#Preenchendo para os rendimentos de renda fixa e dividendos
for linha in rend_isentos.index:
    imagem = encontrar_imagem("botao_novo.png")
    pyautogui.click(pyautogui.center(imagem))
    tipo = rend_isentos.loc[linha , "Tipo de Rendimento"]
    CNPJ = rend_isentos.loc[linha, "CNPJ Formatado"]
    nome = rend_isentos.loc[linha, "Nome da Fonte Pagadora"]
    valor = rend_isentos.loc[linha, "Valor"]
    valor = converter_num_Brasil(str(valor))
    if tipo == "12":
        pyautogui.write(str(tipo))
        pyautogui.press("enter")
        pyautogui.click(x=631, y=611)
        copiar_colar(CNPJ)
        pyautogui.press("tab")
        pyautogui.write(str(nome))
        pyautogui.press("tab")
        pyautogui.write(str(valor))
    elif tipo == "09":
       pyautogui.write(str(tipo))
       pyautogui.press("enter")
       pyautogui.click(x=621, y=583)
       copiar_colar(CNPJ)
       pyautogui.press("tab")
       pyautogui.write(str(nome))
       pyautogui.press("tab")
       pyautogui.write(str(valor)) 
    imagem = encontrar_imagem("botao_ok.png")
    pyautogui.click(pyautogui.center(imagem))
   
    


<span style="font-size: 200%;"><b>Rendimentos Sujeitos à Tributação Exclusiva</b></span>

In [86]:
imagem = encontrar_imagem("rend_trib.png")
pyautogui.click(pyautogui.center(imagem))

for linha in rend_trib_excl.index:
    tipo = rend_trib_excl.loc[linha, "Tipo de Rendimento"]
    CNPJ = rend_trib_excl.loc[linha, "CNPJ Formatado"]
    nome = rend_trib_excl.loc[linha, "Nome da Fonte Pagadora"]
    valor = converter_num_Brasil(str(rend_trib_excl.loc[linha, "Valor"]))
    imagem = encontrar_imagem("botao_novo.png")
    pyautogui.click(pyautogui.center(imagem))
    if tipo == "06":
        pyautogui.write(tipo)
        pyautogui.press("enter")
        pyautogui.click(x=629, y=590)
        copiar_colar(CNPJ)
        pyautogui.press("tab")
        pyautogui.write(nome)
        pyautogui.press("tab")
        pyautogui.write(valor)
    elif tipo == "10":
        pyautogui.write(tipo)
        pyautogui.press("enter")
        pyautogui.click(x=633, y=591)
        copiar_colar(CNPJ)
        pyautogui.press("tab")
        pyautogui.write(nome)
        pyautogui.press("tab")
        pyautogui.write(valor)
    imagem = encontrar_imagem("botao_ok.png")
    pyautogui.click(pyautogui.center(imagem))

        

<span style="font-size: 200%;"><b>Pagamento Efetuado</b></span>

In [87]:
imagem = encontrar_imagem("pag_efetuado.png")
pyautogui.click(pyautogui.center(imagem))

for linha in pagamentos_efetudos.index:
    tipo = pagamentos_efetudos.loc[linha, "Código"]
    CNPJ = pagamentos_efetudos.loc[linha, "CNPJ Formatado"]
    nome = pagamentos_efetudos.loc[linha, "Nome da instituição prestadora do serviço"]
    descricao = pagamentos_efetudos.loc[linha, "Descrição"]
    valor = converter_num_Brasil(str(pagamentos_efetudos.loc[linha, "Valor Pago"]))
    reembolso = converter_num_Brasil(str(pagamentos_efetudos.loc[linha, "Parcela não dedutível/valor reembolsado"]))
    imagem = encontrar_imagem("botao_novo.png")
    pyautogui.click(pyautogui.center(imagem))
    pyautogui.write(tipo)
    pyautogui.press("enter")
    pyautogui.click(x=569, y=483)
    copiar_colar(CNPJ)
    pyautogui.press("tab")
    pyautogui.write(nome)
    pyautogui.press("tab")
    copiar_colar(descricao)
    pyautogui.press("tab")
    pyautogui.write(valor)
    pyautogui.press("tab")
    pyautogui.write(reembolso)
    imagem = encontrar_imagem("botao_ok.png")
    pyautogui.click(pyautogui.center(imagem))

pyautogui.click(x=764, y=125)

<span style="font-size: 200%;"><b>Bens e Direitos</b></span>

In [88]:

#Enquanto não achar a opção de Bens e Direito, a automacão deverá clicar na seta para baixo
while not encontrar_imagem("bens_direitos.png"):
    pyautogui.click(x=438, y=668)
    time.sleep(1) 

imagem = encontrar_imagem("bens_direitos.png")
pyautogui.click(pyautogui.center(imagem))

#Inserindo os dados da planilha Avenue

for linha in avenue.index:
    cod_negociacao = avenue.loc[linha, "Código de Negociação"]
    razao_social = avenue.loc[linha, "Razão Social"]
    ano_anterior = converter_num_Brasil(str(avenue.loc[linha, "Valor Ano Anterior"]))
    ano_atual = converter_num_Brasil(str(avenue.loc[linha, "Valor Atual"]))
    grupo = avenue.loc[linha, "Grupo"]
    codigo = avenue.loc[linha, "Código"]
    pais = avenue.loc[linha, "País"]
    quantidade = converter_num_Brasil(str(avenue.loc[linha, "Quantidade"]))
    taxa = converter_num_Brasil(str(avenue.loc[linha, "Taxa"]))
    preco = converter_num_Brasil(str(avenue.loc[linha, "Preço Médio"]))
    
    imagem = encontrar_imagem("botao_novo.png")
    pyautogui.click(pyautogui.center(imagem))

    pyautogui.write(grupo)
    pyautogui.press("enter")
    pyautogui.write(codigo)
    pyautogui.press("enter")
    pyautogui.press("delete", presses=15)
    pyautogui.write(pais)
    pyautogui.press("tab")
    pyautogui.click(x=557, y=643)
    pyautogui.write(f"{quantidade} do ativo {razao_social} adquirido a um valor medio de US$ {preco} com taxa de R$ {taxa} na corretora Avenue totalizando R$ {ano_atual}")
    imagem = encontrar_imagem("botao_nao.png")
    pyautogui.click(pyautogui.center(imagem))
    pyautogui.press("tab")
    pyautogui.write(ano_anterior)
    pyautogui.press("tab")
    pyautogui.write(ano_atual)
    imagem = encontrar_imagem("botao_ok.png")
    pyautogui.click(pyautogui.center(imagem))

#Inserido os dados dos ativos barsileiros presente na planilha de Posição Ativos

for linha in posicao_ativos_df.index:
    cod_negociacao = posicao_ativos_df.loc[linha, "Código de Negociação"]
    razao_social = posicao_ativos_df.loc[linha, "Razão Social"]
    ano_anterior = converter_num_Brasil(str(posicao_ativos_df.loc[linha, "Valor Ano Anterior"]))
    ano_atual = converter_num_Brasil(str(posicao_ativos_df.loc[linha, "Valor Atualizado"]))
    grupo = posicao_ativos_df.loc[linha, "Grupo"]
    codigo = posicao_ativos_df.loc[linha, "Código"]
    quantidade = converter_num_Brasil(str(posicao_ativos_df.loc[linha, "Quantidade"]))
    preco = converter_num_Brasil(str(posicao_ativos_df.loc[linha, "Preço Médio"]))
    CNPJ = posicao_ativos_df.loc[linha, "CNPJ Formatado"]
    
    imagem = encontrar_imagem("botao_novo.png")
    pyautogui.click(pyautogui.center(imagem))

    if grupo == "04" and codigo == "04":
        pyautogui.write(grupo)
        pyautogui.press("enter")
        pyautogui.write(codigo)
        pyautogui.press("enter")
        pyautogui.press("delete", presses=15)
        pyautogui.write("Brasil")
        pyautogui.press("tab")
        pyautogui.click(x=575, y=674)
        pyautogui.write(f"{quantidade} do ativo {razao_social} adquirido a um valor medio de R$ {preco}  na corretora BTG Pactual totalizando R$ {ano_atual}")
        imagem = encontrar_imagem("botao_sim.png")
        pyautogui.click(pyautogui.center(imagem))
        pyautogui.press("tab")
        pyautogui.write(cod_negociacao)
        pyautogui.press("tab")
        pyautogui.write(ano_anterior)
        pyautogui.press("tab")
        pyautogui.write(ano_atual)
        imagem = encontrar_imagem("botao_ok.png")
        pyautogui.click(pyautogui.center(imagem))

    else:
        pyautogui.write(grupo)
        pyautogui.press("enter")
        pyautogui.write(codigo)
        pyautogui.press("enter")
        pyautogui.press("delete", presses=15)
        pyautogui.write("Brasil")
        pyautogui.press("tab")
        pyautogui.click(x=572, y=640)
        copiar_colar(CNPJ)
        pyautogui.press("tab")
        pyautogui.write(f"{quantidade} do ativo {razao_social} adquirido a um valor medio de R$ {preco}  na corretora BTG Pactual totalizando R$ {ano_atual}")
        imagem = encontrar_imagem("botao_sim.png")
        pyautogui.click(pyautogui.center(imagem))
        pyautogui.press("tab")
        pyautogui.write(cod_negociacao)
        pyautogui.press("tab")
        pyautogui.write(ano_anterior)
        pyautogui.press("tab")
        pyautogui.write(ano_atual)
        imagem = encontrar_imagem("botao_ok.png")
        pyautogui.click(pyautogui.center(imagem))

#Inserindo os dados da planilha Bens e Direitos

for linha in bens_direitos.index:
    razao_social = bens_direitos.loc[linha, "Razão Social"]
    ano_anterior = converter_num_Brasil(str(bens_direitos.loc[linha, "Valor Ano Anterior"]))
    ano_atual = converter_num_Brasil(str(bens_direitos.loc[linha, "Valor Atual"]))
    grupo = bens_direitos.loc[linha, "Grupo"]
    codigo = bens_direitos.loc[linha, "Código"]
    CNPJ = bens_direitos.loc[linha, "CNPJ Formatado"]
    pais = bens_direitos.loc[linha, "Localização/País"]
    discriminacao = bens_direitos.loc[linha, "Discriminação"]
    
    imagem = encontrar_imagem("botao_novo.png")
    pyautogui.click(pyautogui.center(imagem))

    pyautogui.write(grupo)
    pyautogui.press("enter")
    pyautogui.write(codigo)
    pyautogui.press("enter")
    pyautogui.press("delete", presses=15)
    pyautogui.write(pais)
    pyautogui.press("enter")
    if grupo == "99" and codigo == "99":
        copiar_colar(discriminacao)
        pyautogui.press("enter")
        pyautogui.write(ano_anterior)
        pyautogui.press("tab")
        pyautogui.write(ano_atual)
        imagem = encontrar_imagem("botao_ok.png")
        pyautogui.click(pyautogui.center(imagem))
        continue
    copiar_colar(CNPJ)
    pyautogui.press("enter")
    copiar_colar(discriminacao)
    pyautogui.press("enter")
    pyautogui.write(ano_anterior)
    pyautogui.press("tab")
    pyautogui.write(ano_atual)
    imagem = encontrar_imagem("botao_ok.png")
    pyautogui.click(pyautogui.center(imagem))

pyautogui.alert("Código finalizado. Pode voltar a usar seu computador")


'OK'